In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


# Helper methods

In [ ]:
def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')


#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')


# Group 1 Cases

In [ ]:
cyrillic_group = load_csv_dataset('CSVs of edge cases - for F/Group 1 - Filter Using Match Date/Cyrillic_Guys_Multi_found.csv')

multiple_names_working = load_csv_dataset('CSVs of edge cases - for F/Group 1 - Filter Using Match Date/multiple_names_working.csv')

multiple_names_large_dataset_0 = load_csv_dataset('CSVs of edge cases - for F/Group 1 - Filter Using Match Date/multiple_names_used_large_dataset.csv')

online_lookup_reqd = load_csv_dataset('CSVs of edge cases - for F/Group 1 - Filter Using Match Date/Online lookup required - multiple names found.csv')

match_date_filter_guys = load_csv_dataset('CSVs of edge cases - for F/Group 1 - Filter Using Match Date/match_date_multi_guys.csv')


In [ ]:
#functions i need 
from fuzzywuzzy import process
from Levenshtein import distance

#tokens match one

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches


def vet_tokens_names(name, player):
    player_original = player.copy()
    original_name_tokens = name.split(' ')
    #print(original_name_tokens)
    for player_individual in player:
        #print(player)
        this_player_tokens = player_individual.split(' ')
        
        for token in original_name_tokens:
            if token == '':
                0==0
            else:
                first_letter = token[0]
                #print(initial_token)
                token_found = False
                for token_player in this_player_tokens:
                    if token_player.startswith(first_letter):
                        
                        if Levenshtein.distance(token_player, token) >= len(token_player):
                            #print(f'did not match {token_player} with {token}')
                            0==0
                        else:
                            #print(f'matched {token_player} with {token}')
                            #print(token_player)
                            token_found = True
                            break
                if not token_found:
                    #print(player)
                    if player_individual in player:
                        player.remove(player_individual)
    #print('banana', player)
    if player  == []:
        player = player_original
        #print('razz', player)
        for player_individual in player:
            this_player_tokens = re.split(r'\s+|-|–', player_individual)
            #print('apple', this_player_tokens)
            for token in original_name_tokens:
                if token == '':
                    0==0
                else:
                    first_letter = token[0]
                    token_found = False
                    for token_player in this_player_tokens:
                        if token_player.startswith(first_letter):
                            #print('pine', token_player)
                            if Levenshtein.distance(token_player, token) >= len(token_player):
                                #print(f'did not match {token_player} with {token}')
                                0==0
                            else:
                                #print(f'matched {token_player} with {token}')
                                #print(token_player)
                                token_found = True
                                break
                if not token_found:
                    player.remove(player_individual)
                    break

    #print(player)
    return player

def jersey_has_initial(string):
    initials = set("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
    for i in range(len(string)):
        if string[i] in initials:
            # Check if it's not the last character in the string
            if i < len(string) - 1:
                # Check if it's followed by a space or is the last character
                if string[i + 1] == ' ' or i == len(string) - 1:
                    return True
            # Check if it's the last character in the string
            elif i == len(string) - 1:
                return True
    return False

# def standalone_initials(input_string):
#     # Split the string by spaces
#     words = input_string.split()
    
#     # Initialize a set to store standalone initials
#     standalone_initials = set()
    
#     # Iterate through each word
#     for word in words:
#         # If the word is a single character, add it to the set
#         if len(word) == 1:
#             standalone_initials.add(word)
#         # If the word is longer than one character, check if it's standalone, then add its initial
#         elif len(word) > 1 and " " not in word:
#             standalone_initials.add(word[0])
    
#     # Convert the set to a list and return
#     return list(standalone_initials)
def standalone_initials(input_string):
    # Split the string by spaces
    words = input_string.split()
    
    # Initialize a set to store standalone initials
    standalone_initials = set()
    
    # Iterate through each word
    for word in words:
        # If the word is a single character, add it to the set
        if len(word) == 1:
            standalone_initials.add(word)
        # If the word is longer than one character and doesn't contain spaces,
        # add its initial to the set
        elif len(word) == 2 and " " not in word:
            standalone_initials.add(word[0])
    
    # Convert the set to a list and return
    return list(standalone_initials)


def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}

#MATCH DATE FILTER 
def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''

#FINDING MONEY INFO USSING PLAYER NAME
def tryToFindPlayerMoney(
        DATABASE_RETURN_MESSAGE, #what the DB said when trying to find money info
        NAME_TO_FIND, #the name it found in the DB (name_match[0] originally)
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN, #IF TRUE, SALARIES. OTHERWISE VALUES
        ORIGINAL_SEARCH_NAME #name_match[2]
        ):

    if((DATABASE_RETURN_MESSAGE == "not in DB any of 3 seasons.") or (DATABASE_RETURN_MESSAGE == "asswipe was in DB in before or after season")):
        print('newwwnewnenwenewnwenwenwew', DATABASE_RETURN_MESSAGE)
        
        if DATABASE_RETURN_MESSAGE == 'asswipe was in DB in before or after season':
            money_thisplayer = [0, 'Was in  DB before or after season', True, True]
            return money_thisplayer
        

        INTERNET_LOOKUP_RESULTING_NAMES = AssemblyHelpers.false_name_match_lookup(ORIGINAL_SEARCH_NAME, NATIONALITY_INPUT_PLAYER)
        NUM_NAMES_REMAINING = len(list(INTERNET_LOOKUP_RESULTING_NAMES))
        print(INTERNET_LOOKUP_RESULTING_NAMES)
        #1 MATCH
        if(NUM_NAMES_REMAINING == 1):
            #FIND INFO USING SELENIUM
            # NAME_FOR_SELENIUM_SEARCH = next(iter(INTERNET_LOOKUP_RESULTING_NAMES))
            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN)
            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
            money_thisplayer = [0, 'Lookup Required', True, False]
            return money_thisplayer
        #MULTIPLE MATCHES
        elif(NUM_NAMES_REMAINING >= 2):
            print('goes to FUDOM')
            #FILTER THE NAMES USING THE DATE OF THE MATCH
            list_to_filter_with_match_date = list(INTERNET_LOOKUP_RESULTING_NAMES)
            #NEW FUNCTION 3 
            money_thisplayer, name_results = filter_using_date_of_match(list_to_filter_with_match_date, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN) 
            return money_thisplayer
        #0 MATCHES
        elif(NUM_NAMES_REMAINING == 0):
            #This is return case 9
            money_thisplayer = [0, 'Online Lookup Required', True, False] 
            return money_thisplayer
    
    else:
        money_thisplayer = [0, 'Online Lookup Required', True, False]
        return money_thisplayer

## Success + Working + returned a set of names

In [ ]:
names_matches_dict = {}

In [ ]:
multiple_matches_df = pd.DataFrame()

In [ ]:
#for index, row in multiple_names_working.iterrows():


for i in range(679, 692):
    status = 'Fail'
    index = i
    row = match_date_filter_guys.iloc[index]
    jersey = row['ORIGINAL JERSEY'] #row['English Name']
    if jersey == 'J Gudmunds son':
        jersey = 'Jóhann Berg Gudmundsson'
    elif jersey == 'Ahmad Al Dhe':
        jersey = 'Ahmad Al-Dhefiri'
    elif jersey == 'A Nagy':
        jersey = 'Ádám Nagy'
    elif jersey == 'Al Pereira':
        jersey = 'Álvaro Pereira'
    elif jersey == 'Lo Chih-en':
        jersey = 'ChihEn Lo'
    elif jersey == 'Faisal Al Harbi':
        jersey = 'Faisal Zayed'
    elif jersey == 'Ch Rubio':
        jersey = 'Jesús Rubio'
    elif jersey == 'M-l Büchel':
        jersey = 'M Büchel'
    elif jersey == 'O Romero':
        jersey = 'Ó Romero'
    # if i == 65 or i == 84:
    #     jersey = 'A C González'
    # elif i == 73:
    #     jersey = 'P Quiñonez'
    # elif i == 420:
    #     jersey = 'E Benitez'
    name_set = eval(row['Name(s) Found']) #Match(es) Found
    this_country_code = row['Team Country Code'] #row['Country Code']
    
    #print(f'**{index}** {jersey}')
    season = row['Season']
    name_key = f"{jersey}_{season}"

    if name_key in names_matches_dict.keys():
        correct_names = names_matches_dict[name_key]
        print(f'***{i}*** - {jersey} found in dict. names found: {correct_names}')
        # if i == 47:
        #     correct_names = ['Franklin Lucena']
        if len(correct_names) >= 2:
            multi_list = correct_names
            correct_names = []
            status = 'Multi'
        else:
            status = 'Success'
    else:

        country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
        dataset_nationality_unidecoded = {unidecode(name) for name in dataset_nationality}

        correct_names = []
        for name_single in name_set:
            if check_tokens(jersey, name_single):
                correct_names.append(name_single)
            else:
                0==0
        if len(correct_names) >= 2:
            #print(f'first round matching. multiple names for {jersey}. {correct_names}')
            status = 'Multi'
            multi_list = correct_names
        elif len(correct_names) == 1:
            status = 'Success'
        else:
            correct_names_threshold = threshold_player_match(jersey, dataset_nationality)[0]
            for name_threshold in correct_names_threshold:
                if name_threshold in name_set:
                    correct_names_threshold.remove(name_threshold)

            if len(correct_names_threshold) >= 2:
                for name in correct_names_threshold:
                    if check_tokens(jersey, name_single):
                        correct_names.append(name_single)
                    else:
                        0==0
                if len(correct_names) >= 2:
                    0==0
                elif len(correct_names) == 1:
                    status = 'Success'
                else:
                    last_name = jersey.split(' ')[-1]
                    last_name_threshold_matches = threshold_player_match(last_name, dataset_nationality)[0]
                    if len(last_name_threshold_matches) >= 2:
                        if jersey_has_initial(jersey):
                            name_initials = standalone_initials(jersey)
                            for initial in name_initials:
                                for name in last_name_threshold_matches:
                                    name_tokens = name.split(' ')
                                    for token in name_tokens:
                                        if token != last_name and token.startswith(initial):
                                            correct_names.append(name)
                            
                            if len(correct_names) >= 2:
                                #print(f'threshold matching last names & initials. multiple names for {jersey}. {correct_names}. Game is {row['Match']} on {row['Date']}')
                                status = 'Multi'
                                multi_list = correct_names
                            elif len(correct_names) == 1:
                                status = 'Success'
                            else:
                                print(f'threshold matching last names & initials left nobody. {jersey}')

                    elif len(last_name_threshold_matches) == 1:
                        status = 'Success'
                        correct_names = last_name_threshold_matches
                    else:
                        print(f' {jersey}. no similar last names.')
                    #just use the last name, find all players with the last name. 
                        #then if theres an initial use that. 
                        #then if its still multi, match 

            elif len(correct_names_threshold) == 1:
                correct_names = correct_names_threshold
                status = 'Success'
            else:
                correct_names = vet_tokens_names(jersey, name_set)
                if len(correct_names) >= 2:
                    #print(f'3rd round matching. multiple names for {jersey}. {correct_names}')
                    status = 'Multi'
                    multi_list = correct_names
                elif len(correct_names) == 1:
                    status = 'Success'
                else:
                    print('no results for ', jersey)

        if status == 'Success':
            if jersey_has_initial:
                name_initials = standalone_initials(jersey)
                
                for initial in name_initials:
                    for name in correct_names:
                        found = False
                        name_tokens = name.split(' ')
                        last_name = name_tokens[-1]
                        for token in name_tokens:
                            if token != last_name and token.startswith(initial):
                                #print(token)
                                found = True
                        if found == False:
                            correct_names.remove(name)
                if len(correct_names) >= 2:
                    status = 'Multi'
                elif len(correct_names) == 1:
                    0==0
                else:
                    last_name = jersey.split(' ')[-1]
                    last_name_threshold_matches = threshold_player_match(last_name, dataset_nationality)[0]
                    if len(last_name_threshold_matches) >= 2:
                        if jersey_has_initial(jersey):
                            name_initials = standalone_initials(jersey)
                            for initial in name_initials:
                                for name in last_name_threshold_matches:
                                    name_tokens = name.split(' ')
                                    last_name = name_tokens[-1]
                                    for token in name_tokens:
                                        if token != last_name and token.startswith(initial):
                                            correct_names.append(name)
                            
                            if len(correct_names) >= 2:
                                #print(f'threshold matching last names & initials. multiple names for {jersey}. {correct_names}. Game is {row['Match']} on {row['Date']}')
                                status = 'Multi'
                                multi_list = correct_names
                            elif len(correct_names) == 1:
                                status = 'Success'
                            else:
                                print(f'threshold matching last names & initials left nobody. {jersey}')
        elif status == 'Multi':

            for name in multi_list:
                name_records_seasons = leagues_value[leagues_value['Name'] == name]['Season'].unique()
                #print(season, name, name_records_seasons)
                if not any(abs(s - season) <= 1 for s in name_records_seasons):
                    multi_list.remove(name)
            if len(multi_list) >= 2:
                0==0
            elif len(multi_list) == 1:
                correct_names = multi_list
                status = 'Success'
            else:
                0==0 #that means none of the people u found had records near the year of the game 
        
        if status == 'Success':
            0==0
            print(f'***{i}*** - ✓ - ADDED TO DICT SINGLE - {jersey}:{correct_names}') #correct_names
            #if the name has an initial and theres no token in the single match that starts with that initial
            #you have to call it a fail and print the threshold match of the last name.

            names_matches_dict[name_key] = correct_names

        elif status == 'Multi':

            print(f'***{i}*** - ✓ ---- ADDED TO DICT MULTI ---- {jersey}:{multi_list}') #multi_list
            names_matches_dict[name_key] = multi_list
        else:
            print(f'- FAIL - FAIL - ****{i}**** FAIL – FAIL – ', jersey)

    if status == 'Multi':
        print('filtering using match date...') #if multi, filter using match date 
        filtered_names_match_date = filter_using_date_of_match(multi_list, row['Nationality'], row['Date'], False)[1]
        if type(filtered_names_match_date) == str:
            correct_names = filtered_names_match_date
            status = 'Success'
        else:
            multi_list = filtered_names_match_date


    if status == 'Success':
        new_row = {
            'Match ID': row['Match ID'],
            'Match': row['Match'],
            'Date': row['Date'],
            'Competition': row['Competition'],
            'Nationality': row['Nationality'],
            'Country Code': this_country_code,
            'ORIGINAL JERSEY': jersey,
            'Names_Found': correct_names,
            'Season': season,
            'Status': status
        }
    elif status == 'Multi':
        new_row = {
            'Match ID': row['Match ID'],
            'Match': row['Match'],
            'Date': row['Date'],
            'Competition': row['Competition'],
            'Nationality': row['Nationality'],
            'Country Code': this_country_code,
            'ORIGINAL JERSEY': jersey,
            'Names_Found': multi_list,
            'Season': season,
            'Status': status
        }
    else:
        new_row = {
            'Match ID': row['Match ID'],
            'Match': row['Match'],
            'Date': row['Date'],
            'Competition': row['Competition'],
            'Nationality': row['Nationality'],
            'Country Code': this_country_code,
            'ORIGINAL JERSEY': jersey,
            'Names_Found': name_set,
            'Season': season,
            'Status': 'Fail'
        }

    multiple_matches_df = pd.concat([multiple_matches_df, pd.DataFrame([new_row])], ignore_index=True)
    
        #once single add them to a dataset 
            #match ID
            #match
            #date
            #competition
            #season 
            #nationality
            #country code
            #jersey 
            #names found

            #match case - single or multi or fail 

In [ ]:
row['Match'], row['Date']

In [ ]:
multiple_matches_df[multiple_matches_df['Status'] == 'Multi']

In [ ]:
big_multi_dataset = multiple_matches_df
big_multi_dataset

In [ ]:
big_multi_dataset.to_csv('match_date_multi_guys_MATCHES.csv')

In [ ]:
#STRATEGY - SUCCESS + WORKING

    #for each name in the set 
    #check tokens 
        #if that leaves 1 name, finish

        #if that leaves multiple names 
            #filter using the match date 

        #if that leaves 0 names
            #do the threshold match 

                #if that leaves 1 name, finish


                #if that leaves multiple names 

                    #filter using the match date 

                #if that leaves 0 names
                    #do vet_tokens_names 

                    #if that leaves 1 name, finish

                    #if that leaves multiple names 

                        #filter using the match date 

                    #if that leaves 0 names

                        #????


## Combining success group CSVs

In [ ]:
# Folder path
folder_path = 'Group 1 Working guys'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty DataFrame to store merged data
merged_data = pd.DataFrame()

# Iterate over each CSV file and merge into one DataFrame
for file in csv_files:
    # Read CSV file
    df = pd.read_csv(os.path.join(folder_path, file))
    # Merge with existing data
    merged_data = pd.concat([merged_data, df], ignore_index=True)

# Write merged data to a new CSV file
merged_data.to_csv('working_multiple_names_combined.csv', index=False)

print("Merged data saved to 'working_multiple_names_combined.csv'")